# First-order languages and Set Theory

## First-Order languages

A first-order language is determined by a vocabulary consisting of _variables_, _constants_, _functions_ and _relations_. First, we encode functions and relations.

In [1]:
case class Function(name: String, degree: Int)

case class Relation(name: String, degree: Int)

defined class Function
defined class Relation

We form two kinds of _expressions_, _terms_ and _formulas_ using these. Both of these are defined recursively.

In [2]:
sealed trait Expression

sealed trait Term extends Expression

sealed trait Formula extends Expression

object Term{
    case class Var(name: String) extends Term
    
    case class Const(name: String) extends Term 

    case class Recursive(function: Function, arguments: Vector[Term]) extends Term{
        require(function.degree == arguments.length)
    }
}

object Formula{
    case class Equality(lhs: Term, rhs: Term) extends Formula
    
    case class Atomic(relation: Relation, arguments: Vector[Term]) extends Formula{
        require(relation.degree == arguments.length)
    }

    case class Or(p: Formula, q: Formula) extends Formula

    case class And(p: Formula, q: Formula) extends Formula

    case class Implies(p: Formula, q: Formula) extends Formula

    case class Equivalent(p: Formula, q: Formula) extends Formula

    case class Not(p: Formula) extends Formula

    case class ForAll(x: Term.Var, p: Formula) extends Formula

    case class Exists(x: Term.Var, p: Formula) extends Formula

}

defined trait Expression
defined trait Term
defined trait Formula
defined object Term
defined object Formula

In [3]:
val sum = Function("+", 2)
val succ = Function("succ", 1)
// Some valid terms
import Term._
val n = Var("n")
val m = Var("m")
val one = Const("1")
Recursive(sum, Vector(n, m))
Recursive(succ, Vector(n))
val two = Recursive(succ, Vector(one))

sum: Function = Function("+", 2)
succ: Function = Function("succ", 1)
import Term._

n: Var = Var("n")
m: Var = Var("m")
one: Const = Const("1")
res2_6: Recursive = Recursive(Function("+", 2), Vector(Var("n"), Var("m")))
res2_7: Recursive = Recursive(Function("succ", 1), Vector(Var("n")))
two: Recursive = Recursive(Function("succ", 1), Vector(Const("1")))

In [4]:
// An invalid term
import scala.util.Try
val wrong = Try(Recursive(succ, Vector(one, n)))

import scala.util.Try

wrong: Try[Recursive] = Failure(
  java.lang.IllegalArgumentException: requirement failed
)

In [5]:
// Some formulas
import Formula._
val f1 = Equality(one, two)
val f2 = Equality(n, one)
val leq = Relation("<=", 2)
val f3 = Atomic(leq, Vector(one, two))
val f4 = ForAll(n, f2)
val f5 = Exists(n, f2)
val f6 = ForAll(m, f1)

import Formula._

f1: Equality = Equality(
  Const("1"),
  Recursive(Function("succ", 1), Vector(Const("1")))
)
f2: Equality = Equality(Var("n"), Const("1"))
leq: Relation = Relation("<=", 2)
f3: Atomic = Atomic(
  Relation("<=", 2),
  Vector(Const("1"), Recursive(Function("succ", 1), Vector(Const("1"))))
)
f4: ForAll = ForAll(Var("n"), Equality(Var("n"), Const("1")))
f5: Exists = Exists(Var("n"), Equality(Var("n"), Const("1")))
f6: ForAll = ForAll(
  Var("m"),
  Equality(Const("1"), Recursive(Function("succ", 1), Vector(Const("1"))))
)

The formulas in mathematical notation are

* $1 = 2$
* $n = 1$
* $1 \leq 2$
* $\forall n\ n = 1$
* $\exists n\ n = 1$
* $\exists m\ 1 = 2$

## The language of Sets

The language of sets has:

* a single constant $\phi$
* a single relation $\in$ of degree $2$.

We introduce these as well as a convenience method.

In [6]:
import Term._, Formula._

val phi = Const("empty-set")
val in = Relation("in", 2)

def belongs(x: Term, y: Term) = Formula.Atomic(in, Vector(x, y))

import Term._, Formula._


phi: Const = Const("empty-set")
in: Relation = Relation("in", 2)
defined function belongs

As an example, we define extensionality, i.e. if two sets have the same members, they are equal.
$$\forall x\forall y((\forall a\ (a\in x\iff a \in y)) \implies (x = y))$$

In [7]:
val a = Var("a")
def sameMembers(p: Term, q: Term): Formula = 
    ForAll(a, Equivalent(belongs(a, p), belongs(a, q)))

a: Var = Var("a")
defined function sameMembers

In [8]:
val x = Var("x")
val y = Var("y")
val extensionality = 
   ForAll(x, 
         ForAll(y,
               Implies(sameMembers(x, y), Equality(x, y))))
 

x: Var = Var("x")
y: Var = Var("y")
extensionality: ForAll = ForAll(
  Var("x"),
  ForAll(
    Var("y"),
    Implies(
      ForAll(
        Var("a"),
        Equivalent(
          Atomic(Relation("in", 2), Vector(Var("a"), Var("x"))),
          Atomic(Relation("in", 2), Vector(Var("a"), Var("y")))
        )
      ),
      Equality(Var("x"), Var("y"))
    )
  )
)

## Variables and free variables

* The _value_ of a term can depend on some variables.
* The _truth value_ of a formula can depend on some variables.
* These are called _free variables_.

### Examples

In the language of natural numbers

* $1 = 2$; no free variables
* $n = 1$; free variable n
* $1 \leq 2$; no free variables
* $\forall n\ n = 1$; no free variables
* $\exists n\ n +5 < m$; free variable m
* $\exists m\ 1 = 2$; no free variables



In [9]:
// Variable in a term
def variables(t: Term): Set[Var] = t match {
        case Const(name) => Set()
        case Recursive(function, arguments) =>
            for{
                arg : Term <- arguments.toSet
                x <- variables(arg)
            } yield x
        case Var(name) => Set(Var(name))
    }

defined function variables

In [10]:
variables(Recursive(sum, Vector(n, m)))

res9: Set[Var] = Set(Var("n"), Var("m"))

In [11]:
val p = Recursive(succ, Vector(m))
variables(Recursive(sum, Vector(n, p)))

p: Recursive = Recursive(Function("succ", 1), Vector(Var("m")))
res10_1: Set[Var] = Set(Var("n"), Var("m"))

In [12]:
variables(p)

res11: Set[Var] = Set(Var("m"))

In [13]:
def freeVariables(fmla: Formula) : Set[Var] = fmla match {
    case Equality(lhs, rhs) => 
        variables(lhs) union variables(rhs)
    case And(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Implies(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Equivalent(p, q) => 
        freeVariables(p) union freeVariables(q)
    case Not(p) => 
        freeVariables(p)
    case ForAll(x, p) => 
        freeVariables(p) - x
    case Atomic(relation, arguments) =>
        for {
            arg: Term <- arguments.toSet
            x <- variables(arg)
        } yield x
    case Exists(x, p) => 
        freeVariables(p) - x 
    case Or(p, q) => 
        freeVariables(p) union freeVariables(q)
    }

defined function freeVariables

In [14]:
val fmla1 = Exists(x, belongs(x, y))
val fmla2 = ForAll(y, fmla1)
val fmla3 = belongs(x, y)

fmla1: Exists = Exists(
  Var("x"),
  Atomic(Relation("in", 2), Vector(Var("x"), Var("y")))
)
fmla2: ForAll = ForAll(
  Var("y"),
  Exists(Var("x"), Atomic(Relation("in", 2), Vector(Var("x"), Var("y"))))
)
fmla3: Atomic = Atomic(Relation("in", 2), Vector(Var("x"), Var("y")))

In [15]:
freeVariables(fmla1)
freeVariables(fmla2)
freeVariables(fmla3)

res14_0: Set[Var] = Set(Var("y"))
res14_1: Set[Var] = Set()
res14_2: Set[Var] = Set(Var("x"), Var("y"))

## Set Theory

We have so far:

* the empty set $\phi$.
* the relation $\in$ for sets.
* an axiom saying when two sets are equal.

We also know that we cannot specify sets by arbitrary properties, i.e., like
$$\{p : \text{$p$ is a prime}\}$$.

We shall, however, see that we can define sets such as 
$$\{p \in\mathbb{N} : \text{$p$ is a prime}\}$$
provided we have previously defined (constructed) $\mathbb{N}$.

### Properties

Let $x$ be a variable. A _property of $x$_ is a _formula_ $p = p(x)$ so that $p$ has no free variables other than $x$ ($x$ may also not be a free variable). 

We cannot define sets with _unrestricted comprehension_, i.e., of the form
$$S = \{x : p(x) \}$$
as we get a paradox with the set
$$ S = \{x : x \notin x\}$$

In [16]:
val russell = Not(belongs(x, x))
freeVariables(russell)

russell: Not = Not(Atomic(Relation("in", 2), Vector(Var("x"), Var("x"))))
res15_1: Set[Var] = Set(Var("x"))

### Axiom schema of specification

Given a set $U$ and a property $p= p(x)$ of the variable $x$, there exists a set $S$ given by
$$S = \{x \in U: p(x)\}$$

More formally, fix a formula $p$ with no free variables other than $x$. We have the axiom:
$$\forall U\exists S(x\in S \iff (x \in U \land p(x))$$

In [17]:
// we can implement specification for nice properties
val U = (1 to 20).toSet
val S = U.filter(x => x % 3 == 0) // p(x) = x % 3 == 0 
show(S)

Set(6, 9, 12, 3, 18, 15)


U: Set[Int] = Set(
  5,
  10,
  14,
  20,
  1,
  6,
  9,
  13,
  2,
  17,
  12,
  7,
  3,
  18,
  16,
  11,
  8,
  19,
  4,
  15
)
S: Set[Int] = Set(6, 9, 12, 3, 18, 15)

## Union of a pair of sets

$A \cup B$ is the set whose elements are elements of $A$ and elements of $B$

In [18]:
Set(1, 3, 4) union Set(3, 7, 9)

res17: Set[Int] = Set(1, 9, 7, 3, 4)

The existence of union is given by the _axiom of unordered pairs_

$$\forall A \forall B\exists C(\forall x\ (x\in C \iff (x\in A \lor x\in B )))$$

### Arbitrary union

Given a _set of sets_ $U$, we can take the union of all of these, i.e. there exists
$$\bigcup_{A \in U}A$$
such that $x \in \bigcup_{A \in U}A$ if and only if 
$$\exists A\ (A \in U \land x \in A)$$.

In [19]:
// union of sets
val setOfSets = Set(Set(1, 2), Set(1, 3), Set(7, 8, 23), Set(7))
val bigUnion = setOfSets.flatten

setOfSets: Set[Set[Int]] = Set(Set(1, 2), Set(1, 3), Set(7, 8, 23), Set(7))
bigUnion: Set[Int] = Set(1, 2, 7, 3, 23, 8)

There is an axiom that says $\bigcup_{A \in U}A$ exists (_axiom of unions_)

In [20]:
val big = Set[Any](Set(1, 2), Set(1, 3), Set(7, 8, 23), Set(7))
big contains (Set(1, 2))
(Set[Any](1, 2)).subsetOf(big)
big contains 1

big: Set[Any] = Set(Set(1, 2), Set(1, 3), Set(7, 8, 23), Set(7))
res19_1: Boolean = true
res19_2: Boolean = false
res19_3: Boolean = false

In [21]:
Set[Any](Set(1, 2)).subsetOf(big)

res20: Boolean = true

In [22]:
val anotherBig = Set[Any](1, Set(1), Set(1, 2), Set[Any](2, Set(1, 3)))

anotherBig: Set[Any] = Set(1, Set(1), Set(1, 2), Set(2, Set(1, 3)))

In [23]:
anotherBig contains 1

res22: Boolean = true

In [24]:
Set[Any](1) subsetOf anotherBig
anotherBig contains Set(1)

res23_0: Boolean = true
res23_1: Boolean = true

In [25]:
import scala.collection.mutable.{Set => mSet}
val ss = mSet[Any](mSet(1), mSet(2))

import scala.collection.mutable.{Set => mSet}

ss: collection.mutable.Set[Any] = Set(Set(1), Set(2))

In [26]:
ss += mSet(3, 4)

res25: collection.mutable.Set[Any] = Set(Set(1), Set(2), Set(3, 4))

In [27]:
ss += 1

res26: collection.mutable.Set[Any] = Set(Set(1), Set(2), 1, Set(3, 4))

## Intersections

These do not need a new axiom:

$$A \cap B = \{x \in A : x \in B\}$$

i.e., we are using the property $p = p(x) = x \in B$ with free variable $x$.

__Remark:__ By _axiom of extenstion_ $A \cap B = B \cap A$.

In [28]:
val A = Set(1, 2, 3) 
val B = Set(2,3, 7, 9)
A intersect B

A: Set[Int] = Set(1, 2, 3)
B: Set[Int] = Set(2, 3, 7, 9)
res27_2: Set[Int] = Set(2, 3)

In [29]:
A.filter(x => B contains x)

res28: Set[Int] = Set(2, 3)

### Power Set

The _power set_ of a set $S$ is the set of subsets of $S$,
i.e. 
$$2^S = \{x : x \subset S\}$$

This exists because of the _axiom of power sets_ (which just says it exists).

In [30]:
Set(1, 2, 7).subsets.toSet

res29: Set[Set[Int]] = Set(
  Set(),
  Set(2),
  Set(1, 2),
  Set(2, 7),
  Set(1, 2, 7),
  Set(7),
  Set(1, 7),
  Set(1)
)

In [31]:
Set().subsets.toSet

res30: Set[Set[Nothing]] = Set(Set())

In [32]:
Set().subsets.toSet.size

res31: Int = 1

Formally, we define $A \subset B$ by
$x\in A \implies x \in B$.

The axiom giving existence of power sets says
$$\forall S\exists P (x\in P \iff x\subset S)$$

or in full form
$$\forall S\exists P (x\in P \iff (\forall y\ y\in x\iff y\in S))$$

## Numbers as Sets

Everything in mathematics can be represented as sets. 
As an example, we consider natural numbers. We associate to each number
$n$ a set of _size_ n.

* $0$ is $\phi$.
* $1$ is $\{\phi\}$.
* $2$ is $\{\phi, \{\phi\}\}$.



In general, we define the _successor_ of a set $s$ as
$$succ(s) = s \cup \{s\}$$
and define the set associated to $n$ by
* associate $\varphi$ to $0$.
* if $s$ is associated to $n$, associate $succ(s)$ to $n + 1$.

__Remark:__ $\{s\} = \{y \in 2^s: y = s \}$ 

In [33]:
def succ(s: Set[Any]) : Set[Any] = s union Set(s)
succ(Set(1, 2))

defined function succ
res32_1: Set[Any] = Set(1, 2, Set(1, 2))

In [34]:
def numberSet(n: Int): Set[Any] =
     if (n == 0) Set() else succ(numberSet(n - 1))
numberSet(3)

defined function numberSet
res33_1: Set[Any] = Set(Set(), Set(Set()), Set(Set(), Set(Set())))

In [35]:
(0 to 10).map (n => numberSet(n).size)

res34: collection.immutable.IndexedSeq[Int] = Vector(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10
)

* Talking of _size_ is circular. Instead Cantor defined when two sets have the _same size_. Formally, we say _same cardinality_.

* We say two sets have the same _cardinality_ if there is a one-to-one correspondence between their elements.

* By this definition, _natural numbers_ and _even natural numbers_ have the same cardinality.



## Functions : semi-formal

A function $f: X \to Y$ is given by

* A set $X$, called the domain.
* A set $Y$, called the co-domain.
* For each $x\in X$, we associate $f(x)\in Y$.

__Example:__ We can take $X = Y = \mathbb{N}$ and define $f(n) = 2n$.

### Injective (one-to-one)

A function $f: X \to Y$ is said to be _injective_ if for $a, b\in X$, if $a\neq b$ then $f(a)\neq f(b)$, i.e.
$$\forall a,b\in X, a\neq b\implies f(a)\neq f(b)$$

Equivalently,
$$\forall a, b\in X, (a = b) \lor (f(a) \neq f(b))$$

In [36]:
def isInjective[A, B](domain: Set[A], codomain: Set[B], f: A => B) =
     domain.forall{a => 
        domain.forall{b =>
          (a == b) || (f(a) != f(b))}}

defined function isInjective

In [37]:
isInjective((1 to 10).toSet, (1 to 30).toSet, (n: Int) => 2 * n)

res36: Boolean = true

In [38]:
isInjective((1 to 10).toSet, (1 to 30).toSet, (n: Int) => n/ 2)

res37: Boolean = false

In [39]:
isInjective(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 2, 3 -> 1))
isInjective(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 3, 3 -> 2))

res38_0: Boolean = false
res38_1: Boolean = true

### Range and surjectivity

The _range_ of $f: X \to Y$ is defined as 
$$f(X) = \{f(x): x \in X\}$$

We say $f: X \to Y$ is _surjective_ if $f(X) = Y$.

In [40]:
def range[A, B](domain: Set[A], codomain: Set[B], f: A => B) =
        domain.map(f)
range(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 2, 3 -> 1))
range(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 3, 3 -> 2))

defined function range
res39_1: Set[Int] = Set(1, 2)
res39_2: Set[Int] = Set(1, 3, 2)

In [41]:
def isSurjective[A, B](domain: Set[A], codomain: Set[B], f: A => B) =
    range(domain, codomain, f) == codomain

isSurjective(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 2, 3 -> 1))
isSurjective(Set(1, 2, 3), Set(1, 2, 3), Map(1 ->1, 2 -> 3, 3 -> 2))
isSurjective(Set(1, 2, 3), Set(1, 2), Map(1 ->1, 2 -> 2, 3 -> 1))


defined function isSurjective
res40_1: Boolean = false
res40_2: Boolean = true
res40_3: Boolean = true

### Bijections and Cardinality

* A function $f: X \to Y$ is said to be _bijective_ if it is both injective and surjective.
* Two sets $X$ and $Y$ have the same cardinality if there exists a bijective function $f: X \to Y$.

In [42]:
def isBijective[A, B](domain: Set[A], codomain: Set[B], f: A => B) =
    isSurjective(domain, codomain, f) && isInjective(domain, codomain, f)

defined function isBijective

In [43]:
isBijective(Set(1, 2, 3), Set(2, 4, 6), Map(1 -> 2, 2-> 4, 3 -> 6))

res42: Boolean = true

In [44]:
isBijective(
    Set(1, 2, 3), Set(1, 2, 3, 4, 5, 6), Map(1 -> 2, 2-> 4, 3 -> 6)
)

res43: Boolean = false

In [45]:
isBijective(Set(1, 2, 3), Set(1, 2, 3), Map(1 -> 2, 2-> 1, 3 -> 1))

res44: Boolean = false

### Inverses for bijections

If $f: X \to Y$ is a bijection, then it has an _inverse_ $f^{-1}: Y\to X$ so that
$$\forall x\in X, f^{-1}(f(x)) = x$$
and
$$\forall y\in Y, f(f^{-1}(y)) = y$$
and conversely. Further, $f^{-1}: Y \to X$ is a bijection.

In [46]:
def inverseFunction[A, B](domain: Set[A], codomain: Set[B], f: A => B) ={
    require(isBijective(domain, codomain, f), 
            "can only invert bijections")
    (y: B) => domain.find(x => f(x) == y).get
}.ensuring{
    g =>
      domain.forall(x => g(f(x)) == x) &&
      codomain.forall(y => f(g(y)) == y)
}

inverseFunction(Set(1, 2, 3), Set(1, 2, 3), Map(1 -> 2, 2 ->3, 3 -> 1))

defined function inverseFunction
res45_1: Int => Int = ammonite.$sess.cmd45$Helper$$Lambda$2922/892356471@5781e1f7

In [47]:
Try{
    inverseFunction(Set(1, 2, 3), 
                    Set(1, 2, 3), Map(1 -> 2, 2 ->1, 3 -> 1))
}

res46: Try[Int => Int] = Failure(
  java.lang.IllegalArgumentException: requirement failed: can only invert bijections
)

In [48]:
def inverseMap[A, B](domain: Set[A], codomain: Set[B], f: A => B) ={
    require(isBijective(domain, codomain, f), 
            "can only invert bijections")
    (codomain.map{
     (y: B) => y -> domain.find(x => f(x) == y).get}).toMap
}.ensuring{
    g =>
      isBijective(codomain, domain, g) &&
      domain.forall(x => g(f(x)) == x) &&
      codomain.forall(y => f(g(y)) == y)
}

defined function inverseMap

In [49]:
inverseMap(Set(1, 2, 3), Set(4, 5, 6), Map(1 -> 4, 2 -> 6, 3 -> 5))

res48: Map[Int, Int] = Map(4 -> 1, 5 -> 3, 6 -> 2)

## Cantor's theorem

__Theorem:__ For any set $S$, any function $f: S \to 2^S$ 
is _not_ a surjection. In particular $S$ and $2^S$ do not have the same
cardinality.

In other words, if $f: S\to 2^S$, there exists $y\in 2^S$ 
such that $y\notin f(S)$.

__Proof:__ We define $y\in 2^S$, i.e., $y\subset S$,
which is not in the range by
$$y = \{x \in S: x \notin f(x)\}$$

This is not in $f(S)$ as for all $x_0\in X$, then observe that
$$x_0\in y \iff x_0 \notin f(x_0)$$
so $y\neq f(x_0)$.


In [50]:
def cantor[A](s: Set[A], f: A => Set[A]): Set[A] =
     s.filter{x => !f(x).contains(x)
}.ensuring(t => !range(s, s.subsets.toSet, f).contains(t))

defined function cantor

In [51]:
cantor(Set(1, 2, 3), Map(1 -> Set(1), 2 -> Set(1, 2), 3 -> Set(1, 2, 3)))

res50: Set[Int] = Set()

In [52]:
cantor(Set(1, 2, 3), Map(1 -> Set(1), 2 -> Set(1, 2), 3 -> Set(1, 2)))

res51: Set[Int] = Set(3)

### Hilbert's hotel

This is a hotel with infinitely many rooms, with number $1$, $2$, $3$, $4$, ...


* Suppose all the rooms are occupied and a guest turns up, he can still be accomodated: shift everyone.

* Suppose guests with employee codes 1, 2, 3, 4, ... turn up, all can be accomodated.

* Suppose guests with employee codes all _infinite sequences of 0's and 1's_ turn up, they __cannot__ be accomodated.

### Comparing sets

We have defined what it means for sets $A$ and $B$ to have the same size. We denote this $|A| = |B|$ (for now $|A|$ has no meaning).

__Question:__ When should we say $|A| \leq |B|$?

* If $B \subset A$ we expect $|B| \leq |A|$.
* If $|B| = |C|$ and $|C| \leq |A|$ then $|B| \leq |A|$.

__Definition:__ We say $|A| \leq |B|$ if there is an injective function from $f: A \to B$. 

__Question:__ If $|A| \leq |B|$ and $|B| \leq |A|$, does it follow that $|A| = |B|$?

__Answer:__ Yes. This is the Cantor-Bernstein-Shroeder theorem.

__Question (Trichotomy)__: Given sets $A$ and $B$, is at least one of $|A| \leq |B|$ and $|B| \leq |A|$ true?

__Answer:__ Yes. Follows from the following.

__Proposition:__ There are subsets $C \subset A$ and $D \subset B$ and a bijection from $C$ to $D$ such that either $C = A$ or $D = B$.

That is, there is either a bijection from a subset of $A$ to all of $B$ or from $A$ to a subset of $B$.

In [55]:
def trichotomy[X, Y](A : Set[X], B: Set[Y]) : Map[X, Y] = {
    if (A.isEmpty || B.isEmpty) Map[X,Y]()
    else {
        val x = A.head
        val y = B.head
        trichotomy[X, Y](A - x, B - y) + (x -> y)
    }
}.ensuring{(f : Map[X, Y]) => (f.keySet == A || f.values.toSet == B) && 
           f.keySet.subsetOf(A) &&
           f.values.toSet.subsetOf(B) &&
           isBijective(f.keySet, f.values.toSet, f)
          }

defined function trichotomy

In [56]:
trichotomy(Set(1, 2, 4), Set(1, 4, 5, 7))

res55: Map[Int, Int] = Map(4 -> 5, 2 -> 4, 1 -> 1)

In [57]:
trichotomy(Set(true, false), Set(3, 4, 5))

res56: Map[Boolean, Int] = Map(false -> 4, true -> 3)

In [58]:
trichotomy(Set(1, 2, 4), Set("Hello", "Hi"))

res57: Map[Int, String] = Map(2 -> "Hi", 1 -> "Hello")

In the finite case, this terminates by induction. In general, we have to use _transfinite induction_.

__Definition:__ A set $A$ is said to be _countable_ if $|A| \leq |\mathbb{N}|$. Otherwise we say it is _uncountable_.

__Propn:__ If $A$ is countable either $A$ is finite or $|A| = \mathbb{N}$